In [1]:
from functools import partial
import glob
from multiprocessing import Pool
import numpy as np
import os
import pandas as pd
import pybedtools as pbt
import re
import subprocess
import sys

sys.path.append("/dors/capra_lab/users/fongsl/tools/py_/")
sys.path.append("/dors/capra_lab/users/fongsl/tools/genome/")
import config_readwrite as crw
import chr_functions
import split_filename

In [2]:
name = "/data/hodges_lab/ATAC-STARR_B-cells/bin_human-evolution/config"
config, configfile_name = crw.read_config(name)

SHUF = config["CIS_TRANS_LIFTOVER"]["shuf-all"]
REGIONS = config["CIS_TRANS_LIFTOVER"]["regions"]

CL = "LCL8664"
FP_PATH = config["TF_FOOTPRINTING_JASPAR"][f"{CL}_bindetect"]
FP_RE = config[f"TF_FOOTPRINTING_JASPAR_{CL}"]["FP"]
ID_TAG = config["TF_FOOTPRINTING_JASPAR"]["ID_TAG"]

RE = config["TF_FOOTPRINTING"]["results"]

path, region_file, region = split_filename.split_filename(REGIONS)

0. string split to get TF name
1. intersect regions w footprint files
2. create matrix w/ region_id (should be 17605 regions x 693 archetypes) 

# functions

## step 0 

In [3]:
def get_arch_id(filename):
    """
    return archetype ID w/ chr/int split on first two values
    
    do string formatting
    """
    
    match = re.match(r"([a-z]+)([0-9]+)", filename.split("_")[0], re.I)  
    
    if match:
        items = match.groups()
    arch_id = ("".join(items))
    
    return arch_id

In [4]:
def get_tf_bound_file(f, filename, cell_line):

    tf_bound_file = os.path.join(f, "beds", f"{filename}_{cell_line}_1.8-filter_bound.bed")
    
    return tf_bound_file

In [5]:
def get_tf_paths(path, cell_line, id_tag):
    """
    return dictionary w/ key = archetype id and value = tuple of TF name and full file path.  
    
    1. get all the file names
    2. get arch_id name - string split function
    3. get TF name - str split
    4. get TF FP bound file
    5. make dictionary
    6. return dictionary
    """
    tf_dict ={}
    
    #1
    fs = glob.glob(os.path.join(path, f"*{id_tag}*"))
    print(len(fs))
    for f in fs:
        #2
        filename = os.path.split(f)[1]  # get the file name
        #3
        if "MA" in id_tag:
            arch_id = filename.upper()  
            tf_name = filename.split("_")[0]
        else:
            arch_id=get_arch_id(filename) #FOR ARCHTYPES ONLY
            
            #4
            tf_name = "_".join((filename.split(arch_id)[1]).split("_")[:-1])  # get TF name by splitting string. 

        
        #5
        tf_bound_file = get_tf_bound_file(f, filename, cell_line)  # get path to TF bound.bed file
        #6
        tf_dict[arch_id] = (tf_name, tf_bound_file)
    
    return tf_dict

## step 1 

In [8]:
def intersect_w_fp(regions, fp_name, fp_bed, outdir, cell_line, shuf):
    
    if shuf is True:
        cell_line = f"shuf-{cell_line}"
        
    if os.path.exists(os.path.join(outdir, cell_line)) is False:  # make a directory
        os.mkdir(os.path.join(outdir, cell_line))
        
    out = os.path.join(outdir, cell_line, f"{cell_line}-{fp_name}.bed")
    
    if os.path.exists(out) is False:  # if intersection hasn't already been done. 
        
        bedregions = pbt.BedTool(regions)
    
        bedregions.intersect(pbt.BedTool(fp_bed), wo = True).saveas(out)
        print(out)
    
    return out
    

In [9]:
def make_region_fp_matrix(outdir, cell_line, id_tag, shuf):

    if shuf is True:
        cell_line = f"shuf-{cell_line}"
        
    print(cell_line,"making matrix")
    """
    return dictionary of sample_id keys and pandas dataframes (values) for overlapping TF footprints
    
    1. get all region_x_TF.bed intersection files
    2. get sample_id 
    3. pick which columns and column names to load from file
    4. load file
    5. FILTER for overlaps w/ >5bp (size of TFBS)
    6. add to dictionary
    7. return dictionary
    """
    #1
    fs = glob.glob(os.path.join(outdir, cell_line, f"*{id_tag}*.bed"))
    
    dfs = {}
    for i, f in enumerate(fs):
        
        #2
        path, file, sample_id = split_filename.split_filename(f)
        #4
        usecols = list(np.arange(8))
        usecols.append(21)
        
        colnames = ["#chr", "start", "end", 'region_id', 
                    "#chr_fp", "start_fp", "end_fp", 
                    "fp_id", "overlap"]
        #4
        d = pd.read_csv(f, sep = '\t', header = None, usecols = usecols, names = colnames)
        #5
        d = d.loc[d["overlap"]>=5]
        #6
                
        arch_id = (sample_id.split(".bed")[0]).split("-")[-1]
        d[arch_id] = True
        #7
        dfs[sample_id] = d[["region_id", arch_id]].drop_duplicates()
        
        print(i)
    return dfs

## step 2

In [10]:
def join_fp_matrices(dfs, region_file):
    print("joining")
    
    """
    return pandas dataframe of all fp matrices
    1. open the region file
    2. consecutively join all other region_x_fp intersections
    3. fillin na w/ False
    
    """
    #1
    region_f = pd.read_csv(region_file, sep = '\t')
    
    #2
    for i, v in enumerate(dfs.values()):

        if i == 0:  # only merge the region file the first time
            regions = pd.merge(region_f, v, how = "left", on = "region_id")  
            
        else:
            regions = pd.merge(regions, v, how = "left")  #left merge the region results. 
            
    print(regions.shape)
    
    #3
    regions = regions.fillna(False)
    
    return regions

In [11]:
def check_regions_colnames(f):
    """
    rename columns for region file if needed. 
    """
    
    r = pd.read_csv(f, sep = '\t', nrows=2)  # read the first two lines to check. 
    
    if "region_id" not in list(r):  # if no column names:
        
        r = pd.read_csv(f, sep = '\t', header = None)  # read in dataframe
        r.columns = ["#chr", "start", "end", "region_id"]  # rename columns
    
        print(r.shape)

        r.to_csv(f, sep = '\t', index = False)  # resave dataframe w/ column names

# pipeline function 

In [12]:
def pipeline(outfile, cl, id_tag, regions, re, fp_path, shuf):
    
    if os.path.exists(outfile) is False:
        tfs = get_tf_paths(fp_path, cl, id_tag) ## get all the tf.bed paths

        for k, vs in tfs.items():  # per TF footprint file
            tfname, FP_bed = vs[0], vs[1]

            out = intersect_w_fp(regions, k, FP_bed, re, cl, shuf)  # intersect regions w fp

        dfs = make_region_fp_matrix(re, cl, id_tag, shuf)  # open the intersections as pandas dataframes, keep only FP overlaps > 5

        check_regions_colnames(regions)
        
        regions_df = join_fp_matrices(dfs, regions)

        regions_df.to_csv(outfile, sep = '\t', index = False)

    else:
        print("done!")

# main

In [16]:
SHUFFLED = False
pipeline(FP_RE, CL, ID_TAG, REGIONS, RE, FP_PATH, SHUFFLED)

746
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-TGIF2LX_MA1571.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-POU6F2_MA0793.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-ESRRA_MA0592.3.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-HIC2_MA0738.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-AR_MA0007.3.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-ISX_MA0654.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-BCL6_MA0463.2.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-RARBVAR.3_MA1552.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/

/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-MECOM_MA0029.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-NOTO_MA0710.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-BACH1MAFK_MA0591.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-LHX5_MA1519.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-KLF13_MA0657.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-ATOH1_MA0461.2.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-HIC1_MA0739.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-NR4A2RXRA_MA1147.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results

/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-DMRTC2_MA1479.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-ASCL1VAR.2_MA1631.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-PPARARXRA_MA1148.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-HOXA13_MA0650.2.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-PROX1_MA0794.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-PTF1AVAR.2_MA1619.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-ATOH7_MA1468.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-RAX2_MA0717.1.bed
/data/hodges_lab/ATAC-STARR_B-cel

/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-FOSL2JUNB_MA1138.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-NFATC1_MA0624.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-TAL1TCF3_MA0091.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-DDIT3CEBPA_MA0019.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-STAT5ASTAT5B_MA0519.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-EOMES_MA0800.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-ZIC4_MA0751.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-MEF2D_MA0773.1.bed
/data/hodges_lab/ATAC-STARR_B

/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-EMX1_MA0612.2.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-KLF10_MA1511.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-BARHL2_MA0635.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-MAF_MA1520.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-MZF1_MA0056.2.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-IRF1_MA0050.2.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-OSR1_MA1542.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-JUNB_MA0490.2.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_hu

...

/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-MEOX1_MA0661.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-VSX2_MA0726.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-FOSL1JUN_MA1128.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-RORC_MA1151.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-IRF9_MA0653.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-ONECUT3_MA0757.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-FERD3L_MA1485.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-BATFJUN_MA0462.2.bed
/data/hodges_lab/ATAC-STARR_B-cells/results

/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-ZNF274_MA1592.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-VEZF1_MA1578.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-MSGN1_MA1524.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-THRB_MA1574.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-REL_MA0101.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-PAX1_MA0779.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-SIX2_MA1119.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-TBX20_MA0689.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_

/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-MEF2B_MA0660.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-NR1D2_MA1532.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-ZNF148_MA1653.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-DUX_MA0611.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-PBX2_MA1113.2.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-HOXC8_MA1505.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-MEF2C_MA0497.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-BARX2_MA1471.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/result

/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-HOXB7_MA1501.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-STAT3_MA0144.2.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-HES6_MA1493.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-BARHL1_MA0877.2.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-NFIB_MA1643.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-RXRGVAR.2_MA1556.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-GLIS3_MA0737.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664/LCL8664-FOXJ2_MA0614.1.bed
/data/hodges_lab/ATAC-STARR_B-cells/results/r

686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
(17604, 750)


In [13]:
FP_RE

'/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/LCL8664_1.8-JASPAR_0.05-pval_morethan5bp.bed'

# shuffle

In [14]:
CL_SHUF = f"shuf-{CL}"
FP_RE = os.path.join(RE, f"{CL_SHUF}_1.8-JASPAR_0.05-pval_morethan5bp.bed")
SHUFFLED = True
print(FP_RE)
pipeline(FP_RE, CL, ID_TAG, SHUF, RE, FP_PATH, SHUFFLED)

/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/shuf-LCL8664_1.8-JASPAR_0.05-pval_morethan5bp.bed
746
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
